# Share Analysis

In [ ]:
# Solve SynchronousOnlyOperation error
# https://stackoverflow.com/questions/61926359/django-synchronousonlyoperation-you-cannot-call-this-from-an-async-context-u

import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

### Extract data from db and convert it in a seaborn compatible data source

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from voices.website import models

In [ ]:
shares = models.Share.objects.order_by("timestamp").all()
data = {
    "datetime": [s.timestamp for s in shares],
    "latitude": [s.latitude for s in shares],
    "longitude": [s.longitude for s in shares],
    "day": [s.timestamp.date() for s in shares],
}
df = pd.DataFrame(data)


df["timestamp"] = df.datetime.astype(int) // 1000


# Convert to mercator coordinates (see: https://stackoverflow.com/questions/57178783/how-to-plot-latitude-and-longitude-in-bokeh)
k = 6378137
df["x"] = df["longitude"] * (k * np.pi/180.0)
df["y"] = np.log(np.tan((90 + df["latitude"]) * np.pi/360.0)) * k

df.tail()

### Bokeh 

In [ ]:
from bokeh.plotting import curdoc, figure, show
from bokeh.io import output_notebook
from bokeh.transform import linear_cmap
from bokeh.models import ColumnDataSource
import xyzservices.providers as xyz

output_notebook()
curdoc().theme = 'dark_minimal'

In [ ]:
source = ColumnDataSource(df)
p = figure(
    match_aspect=True, 
    aspect_ratio=3, 
    sizing_mode="stretch_width",
    x_axis_type="mercator", 
    y_axis_type="mercator",
    x_range=(-2000000, 2000000), 
    y_range=(1000000, 7000000),
)
p.add_tile(xyz.Stamen.TonerBackground)

In [ ]:
cmap = linear_cmap(
    "timestamp", 
    'Viridis256', 
    low=df.timestamp.min(),
    high=df.timestamp.max(),
)

p.circle(
    source=source, 
    x="x", 
    y="y", 
    color=cmap,
    size=10,
) 

In [ ]:
show(p)

### Matplotlib/seaborn plot

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%matplotlib ipympl
sns.scatterplot(data=data, x="longitude", y="latitude", hue="day")

In [ ]:
# Decomment to custoomize and update data if in ipympl mode
# sns.scatterplot(data=data, x="latitude", y="longitude", hue="day")